In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine


In [4]:
#Loading csv files

SingleFamilyRes_prices_file = "City_Zhvi_SingleFamilyResidence.csv"
SingleFamilyRental_prices_file = "City_Zri_SingleFamilyResidenceRental.csv"
#city_prices_2_file = "Sale_Prices_City 2.csv"

In [5]:
SFRes_prices_df = pd.read_csv(SingleFamilyRes_prices_file)
SFRes_prices_df.head()

,Unnamed: 0,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,0,6181,0,New York,City,NY,NY,New York-Newark-Jersey City,Queens County,208545.0,...,672433.0,671924.0,671423.0,670719.0,669974.0,669118.0,668736.0,668740.0,668581.0,668030.0
1,1,12447,1,Los Angeles,City,CA,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,192855.0,...,745290.0,746729.0,748924.0,751756.0,755716.0,759279.0,764877.0,770853.0,779717.0,788751.0
2,2,39051,2,Houston,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,95018.0,...,189803.0,190437.0,191052.0,191483.0,192124.0,192620.0,193202.0,193427.0,193991.0,194986.0
3,3,17426,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,126867.0,...,226322.0,226635.0,226796.0,226645.0,226505.0,226430.0,226454.0,226727.0,227077.0,227605.0
4,4,6915,4,San Antonio,City,TX,TX,San Antonio-New Braunfels,Bexar County,94406.0,...,183622.0,184246.0,184831.0,185752.0,186401.0,187159.0,187339.0,187886.0,188055.0,188650.0


In [6]:
#Single Family Residence Sale prices from June 2018 to May 2019

SFRes_prices_2019_df  = SFRes_prices_df.loc[:, ['RegionID', 'RegionName','State', 'SizeRank','CountyName',
                                                '2018-06-30','2018-07-31','2018-08-31','2018-09-30','2018-10-31','2018-11-30',
                                                '2018-12-31','2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31']]
#SFRes_prices_2019_df.rename(columns = {'RegionName':'StateName'}, inplace = True)

SFRes_prices_2019_df.rename(columns = {'2018-06-30': '2018-06','2018-07-31':'2018-07',
                                       '2018-08-31':'2018-08','2018-09-30':'2018-09',
                                      '2018-10-31':'2018-10','2018-11-30':'2018-11',
                                       '2018-12-31':'2018-12','2019-01-31':'2019-01',
                                       '2019-02-28':'2019-02','2019-03-31':'2019-03',
                                       '2019-04-30':'2019-04','2019-05-31':'2019-05'
                                      }, inplace = True)

SFRes_prices_2019_df

,RegionID,RegionName,State,SizeRank,CountyName,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05
0,6181,New York,NY,0,Queens County,656416.0,659558.0,662560.0,665349.0,667553.0,669968.0,671832.0,672878.0,673346.0,673317.0,673344.0,673041.0
1,12447,Los Angeles,CA,1,Los Angeles County,737760.0,740856.0,743248.0,745210.0,746402.0,747926.0,747894.0,748142.0,745590.0,743487.0,742025.0,743137.0
2,39051,Houston,TX,2,Harris County,181196.0,181918.0,182830.0,183934.0,184797.0,185652.0,186471.0,187830.0,188870.0,189345.0,189229.0,189317.0
3,17426,Chicago,IL,3,Cook County,222371.0,223032.0,223516.0,224228.0,224958.0,225668.0,225964.0,225584.0,225392.0,225482.0,225962.0,226217.0
4,6915,San Antonio,TX,4,Bexar County,174341.0,175260.0,176101.0,176433.0,177051.0,177409.0,178802.0,179812.0,181294.0,181723.0,182448.0,182884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27321,52758,Kerens,WV,29449,Randolph County,73590.0,74249.0,74198.0,73169.0,71680.0,70865.0,70418.0,71115.0,71955.0,72911.0,72906.0,73018.0
27322,234408,Jolly,TX,29449,Clay County,142269.0,142212.0,142144.0,142602.0,143298.0,144106.0,145094.0,146296.0,147329.0,147741.0,148134.0,148985.0
27323,16604,May,OK,29449,Harper County,73591.0,73756.0,74307.0,74850.0,75003.0,74860.0,74699.0,74919.0,75174.0,75488.0,75871.0,76035.0
27324,44752,Coppock,IA,29449,Henry County,46646.0,46327.0,46138.0,45939.0,45591.0,45970.0,45969.0,46064.0,45693.0,45763.0,45742.0,45631.0


In [7]:
# add new column with average sale for each state

SFRes_prices_2019_df['Average Sale']  = SFRes_prices_2019_df[['2018-06', '2018-07','2018-08','2018-09','2018-10','2018-11','2018-12','2019-01','2019-02','2019-03','2019-04','2019-05']].mean(axis=1)
# display the dataframe
SFRes_prices_2019_df

,RegionID,RegionName,State,SizeRank,CountyName,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,Average Sale
0,6181,New York,NY,0,Queens County,656416.0,659558.0,662560.0,665349.0,667553.0,669968.0,671832.0,672878.0,673346.0,673317.0,673344.0,673041.0,668263.500000
1,12447,Los Angeles,CA,1,Los Angeles County,737760.0,740856.0,743248.0,745210.0,746402.0,747926.0,747894.0,748142.0,745590.0,743487.0,742025.0,743137.0,744306.416667
2,39051,Houston,TX,2,Harris County,181196.0,181918.0,182830.0,183934.0,184797.0,185652.0,186471.0,187830.0,188870.0,189345.0,189229.0,189317.0,185949.083333
3,17426,Chicago,IL,3,Cook County,222371.0,223032.0,223516.0,224228.0,224958.0,225668.0,225964.0,225584.0,225392.0,225482.0,225962.0,226217.0,224864.500000
4,6915,San Antonio,TX,4,Bexar County,174341.0,175260.0,176101.0,176433.0,177051.0,177409.0,178802.0,179812.0,181294.0,181723.0,182448.0,182884.0,178629.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27321,52758,Kerens,WV,29449,Randolph County,73590.0,74249.0,74198.0,73169.0,71680.0,70865.0,70418.0,71115.0,71955.0,72911.0,72906.0,73018.0,72506.166667
27322,234408,Jolly,TX,29449,Clay County,142269.0,142212.0,142144.0,142602.0,143298.0,144106.0,145094.0,146296.0,147329.0,147741.0,148134.0,148985.0,145017.500000
27323,16604,May,OK,29449,Harper County,73591.0,73756.0,74307.0,74850.0,75003.0,74860.0,74699.0,74919.0,75174.0,75488.0,75871.0,76035.0,74879.416667
27324,44752,Coppock,IA,29449,Henry County,46646.0,46327.0,46138.0,45939.0,45591.0,45970.0,45969.0,46064.0,45693.0,45763.0,45742.0,45631.0,45956.083333


In [8]:
SFRental_prices_df = pd.read_csv(SingleFamilyRental_prices_file)
SFRental_prices_df.head()

,Unnamed: 0,RegionID,RegionName,State,Metro,CountyName,SizeRank,2010-11,2010-12,2011-01,...,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
0,0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,1751.0,1760.0,1773.0,...,2248,2255,2284,2318,2345,2349,2347,2345,2334,2327
1,1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,2310.0,2313.0,2307.0,...,3016,3025,3057,3098,3130,3138,3141,3141,3142,3142
2,2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,1260.0,1245.0,1235.0,...,1454,1457,1469,1482,1491,1493,1494,1495,1495,1496
3,3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,1500.0,1500.0,1489.0,...,1559,1564,1592,1624,1650,1653,1653,1654,1652,1652
4,4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,1074.0,1054.0,1041.0,...,1285,1288,1293,1300,1306,1310,1314,1319,1326,1333


In [9]:
#Single Family Residence Rental prices from June 2018 to May 2019

SFRental_prices_df  = SFRental_prices_df.loc[:, ['RegionID', 'RegionName','State', 'SizeRank','CountyName',
                                                '2018-06','2018-07','2018-08','2018-09','2018-10','2018-11',
                                                '2018-12','2019-01','2019-02','2019-03','2019-04','2019-05']]


SFRental_prices_df

,RegionID,RegionName,State,SizeRank,CountyName,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05
0,6181,New York,NY,1,Queens County,2255,2251,2249,2248,2255,2284,2318,2345,2349,2347,2345,2334
1,12447,Los Angeles,CA,2,Los Angeles County,3026,3023,3019,3016,3025,3057,3098,3130,3138,3141,3141,3142
2,39051,Houston,TX,3,Harris County,1461,1459,1457,1454,1457,1469,1482,1491,1493,1494,1495,1495
3,17426,Chicago,IL,4,Cook County,1563,1560,1559,1559,1564,1592,1624,1650,1653,1653,1654,1652
4,6915,San Antonio,TX,5,Bexar County,1285,1285,1286,1285,1288,1293,1300,1306,1310,1314,1319,1326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,55400,Borden,IN,12409,Clark County,994,994,994,994,998,997,990,975,967,968,972,975
12409,206163,Summit,OK,12410,Muskogee County,619,619,620,620,619,618,621,625,630,633,637,638
12410,234282,Indian Lake,TX,12411,Cameron County,736,731,728,727,727,729,734,742,750,753,755,753
12411,236915,Ranchitos Las Lomas,TX,12412,Webb County,1177,1187,1190,1191,1195,1200,1203,1201,1199,1199,1199,1199


In [10]:
# add new column with average rent for the each region in the county

SFRental_prices_df['Average Rent']  = SFRental_prices_df[['2018-06','2018-07','2018-08','2018-09','2018-10','2018-11',
                                                '2018-12','2019-01','2019-02','2019-03','2019-04','2019-05']].mean(axis=1)
# display the dataframe
SFRental_prices_df

,RegionID,RegionName,State,SizeRank,CountyName,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,Average Rent
0,6181,New York,NY,1,Queens County,2255,2251,2249,2248,2255,2284,2318,2345,2349,2347,2345,2334,2298.333333
1,12447,Los Angeles,CA,2,Los Angeles County,3026,3023,3019,3016,3025,3057,3098,3130,3138,3141,3141,3142,3079.666667
2,39051,Houston,TX,3,Harris County,1461,1459,1457,1454,1457,1469,1482,1491,1493,1494,1495,1495,1475.583333
3,17426,Chicago,IL,4,Cook County,1563,1560,1559,1559,1564,1592,1624,1650,1653,1653,1654,1652,1606.916667
4,6915,San Antonio,TX,5,Bexar County,1285,1285,1286,1285,1288,1293,1300,1306,1310,1314,1319,1326,1299.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12408,55400,Borden,IN,12409,Clark County,994,994,994,994,998,997,990,975,967,968,972,975,984.833333
12409,206163,Summit,OK,12410,Muskogee County,619,619,620,620,619,618,621,625,630,633,637,638,624.916667
12410,234282,Indian Lake,TX,12411,Cameron County,736,731,728,727,727,729,734,742,750,753,755,753,738.750000
12411,236915,Ranchitos Las Lomas,TX,12412,Webb County,1177,1187,1190,1191,1195,1200,1203,1201,1199,1199,1199,1199,1195.000000


In [11]:
#checking for any duplicates in df(s)

any(SFRes_prices_2019_df.duplicated())

False

In [12]:
any(SFRental_prices_df.duplicated())

False

Create Database Connection

In [13]:
connection_string = "postgres:postgres@localhost:5432/SingleFamHouse_SaleRentPrice_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
#connection_string = "postgres:karlarobles@localhost:5432/karlarobles"
#engine = create_engine(f'postgresql://{connection_string}')

In [17]:
# Checking the tables 
engine.table_names()

/var/folders/mc/nvjx38yn6j14pqksc9yx1pk40000gn/T/ipykernel_17786/4058620744.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Use pandas to load csv converted DataFrame into database

SFRes_prices_2019_df.to_sql(name='sfhomesales', con=engine, if_exists='append', index=False)

In [ ]:
#Use pandas to load csv converted DataFrame into database

SFRental_prices_df.to_sql(name='sfhomerental', con=engine, if_exists='append', index=False)

In [ ]:
# ### Confirm data has been added by querying the statesales table

pd.read_sql_query('select * from sfhomesales', con=engine).head()

In [ ]:
### Confirm data has been added by querying the citysales table

pd.read_sql_query('select * from sfhomerental', con=engine).head()

In [ ]:
#Finding the sale price and Rental price for a region in a State.
#Finding the sale price and Rental price for a region in a State.

pd.read_sql_query('''select distinct sh."RegionName" ,sh."RegionID",sh."State",cast(round(cast(sh."Average Sale" as numeric),2) as FLOAT) as "Aversage Sale" ,cast(round(cast(sr."Average Rent" as numeric),2) as FLOAT) as "Average Rent" from sfhomesales  sh, sfhomerental sr where sh."RegionName"=sr."RegionName" and sh."State" = sr."State" and sh."State"='NY' ''', con=engine)
                                                 
                                                 

In [ ]:
#Finding the sale price and Rental price for a region in a State.

table1=pd.read_sql_query('''select distinct sh."RegionName" ,sh."RegionID",sh."State",cast(round(cast(sh."Average Sale" as numeric),2) as FLOAT) as "Aversage Sale" ,cast(round(cast(sr."Average Rent" as numeric),2) as FLOAT) as "Average Rent" from sfhomesales  sh, sfhomerental sr where sh."RegionName"=sr."RegionName" and sh."State" = sr."State" and sh."State"='NY' ''', con=engine)
                                                 

In [ ]:
# Top 5 rent in sale NY
#top5sale=table1.sort_values(by=['Aversage Sale'], ascending=False).head(5)

#fig = plt.figure()
#ax = fig.add_axes([0,0,1,1])
#langs = top5sale['RegionName'].tolist()
#students = top5sale['Aversage Sale'].tolist()
#ax.bar(langs,students)
#plt.savefig("/Users/karlarobles/House_Hunters_Project2/ImagesHouseHunters/Top5inNY.png")
#plt.show()

In [ ]:
# mortage
# P = L[c(1 + c)^n]/[(1 + c)^n - 1], where "L" is the loan value,
# "n" is the total number of payments over the life of the loan 
# and "c" is the interest rate for a single payment period.

# for L is value of the house
# Payments= 30 years 12 months = 360
# interest 0.04 per year or .0033 monthly

#paying half of L

mortagetable20 = table1
#mortagetable3 = table1

c=0.0033;
n=30*12;

mortagetable20['mortage']=(table1['Aversage Sale']*.8)*[(c*(1+c)**(n))/((1+c)**n-1)]
#mortagetable3['mortage']=(table1['Aversage Sale']*.97)*[(c*(1+c)**(n))/((1+c)**n-1)]

In [ ]:
#mortge vs rent in highest mortage cities
top5mortage=mortagetable20.sort_values('mortage', ascending=False).head(10)

X = top5mortage['RegionName'].tolist()
#X = top5mortage['State'].tolist()
Y =top5mortage['Average Rent'].tolist()
Z = top5mortage['mortage'].tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, Y, 0.4, label = 'Monthly Rent')
plt.bar(X_axis + 0.2, Z, 0.4, label = 'Monthly Mortage payment')
plt.xticks(rotation = 45)
  
plt.xticks(X_axis, X)
plt.xlabel("Region")
plt.ylabel("Thousand dollars")
plt.title("Top 10 most expensive Mortage/Rent payment w/20% DP")
plt.legend()
plt.savefig("/Users/karlarobles/House_Hunters_Project2/ImagesHouseHunters/Top10States.png")
plt.show()

In [ ]:
mortagetable3 = table1

c=0.0033;
n=30*12;


mortagetable3['mortage']=(table1['Aversage Sale']*.97)*[(c*(1+c)**(n))/((1+c)**n-1)]

In [ ]:
#mortge vs rent in highest mortage cities


top5mortage=mortagetable3.sort_values('mortage', ascending=False).head(10)

X = top5mortage['RegionName'].tolist()
#X = top5mortage['State'].tolist()
Y =top5mortage['Average Rent'].tolist()
Z = top5mortage['mortage'].tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, Y, 0.4, label = 'Monthly Rent')
plt.bar(X_axis + 0.2, Z, 0.4, label = 'Monthly Mortage payment w/3% DP')
plt.xticks(rotation = 45)
  
plt.xticks(X_axis, X)
plt.xlabel("Region")
plt.ylabel("Thousand dollars")
plt.title("Top 10 most expensive Mortage/Rent payment ")
plt.legend()
plt.savefig("../ImagesHouseHunters/Top10States.png")
plt.show()

In [ ]:
# mortage vs rent in highest rent cities
top5rent=mortagetable.sort_values('Average Rent', ascending=False).head(5)

X = top5rent['RegionName'].tolist()
Y =top5rent['Average Rent'].tolist()
Z = top5rent['mortage'].tolist()
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, Y, 0.4, label = 'Rent')
plt.bar(X_axis + 0.2, Z, 0.4, label = 'Mortage')
plt.xticks(rotation = 45)
  
plt.xticks(X_axis, X)
plt.xlabel("Region")
plt.ylabel("Thousand dollars")
plt.title("Top 5 most expensive Mortage/Rent payment ")
plt.legend()
plt.savefig("../ImagesHouseHunters/Mortgagevsrent.png")
plt.show()

In [ ]:
#top5mean= pd.read_sql_query('select * from sfhomesales', con=engine).groupby('State').mean().head(5)
#top5rent=mortagetable.sort_values('Average Rent', ascending=False).head(5)
mean= pd.read_sql_query('select * from sfhomesales', con=engine).groupby('State').mean()

In [ ]:
mean

In [ ]:
top5mean=mean.sort_values('Average Sale',ascending=False).head(5)
#top5rent=mortagetable.sort_values('Average Rent', ascending=False).head(5)

In [ ]:
top5mean

In [ ]:
# Plot AK
x = ['2018-06','2018-07','2018-08','2018-09','2018-10','2018-11','2018-12','2019-01','2019-02','2019-03','2019-04','2019-05']
yDC = top5mean.iloc[[0]][x].values[0].tolist()
yHI = top5mean.iloc[[1]][x].values[0].tolist()
yCA = top5mean.iloc[[2]][x].values[0].tolist()
yMA = top5mean.iloc[[3]][x].values[0].tolist()
yNJ = top5mean.iloc[[4]][x].values[0].tolist()

plt.plot(x,yDC)
plt.plot(x,yHI)
plt.plot(x,yCA)
plt.plot(x,yMA)
plt.plot(x,yNJ)
plt.xticks(rotation = 45)
plt.legend(['DC', 'HI','CA','MA','NJ'])
plt.ylabel("Thousand dollars")
plt.title("Rent Price over time on the top 5 monst expensive States ")
plt.savefig("/Users/karlarobles/House_Hunters_Project2/ImagesHouseHunters/Top5States.png")
plt.show()